### Funções com Classes

In [1]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float) -> None:
        self.nome = nome
        self.idade = idade
        self.peso = peso


adriano = Pessoa("Adriano", 30, 80)

In [2]:
print(adriano.peso)
print(adriano.idade)
print(adriano.nome)

80
30
Adriano


### Funções com Pydantic

In [3]:
from pydantic import BaseModel

class pydPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

adriano = pydPessoa(nome="Adriano", idade=30, peso=80)
adriano

pydPessoa(nome='Adriano', idade=30, peso=80.0)

In [4]:
from typing import List

class pydAsimovTeam(BaseModel):
    funcionarios : List[pydPessoa]

pydAsimovTeam(funcionarios = [pydPessoa(nome='Rafael', idade=30, peso=68)])

pydAsimovTeam(funcionarios=[pydPessoa(nome='Rafael', idade=30, peso=68.0)])

-----------------------------------------------------

In [5]:
import json

def obter_temperatura_atual(local, unidade="celsius"):
    if "sao paulo" in local.lower():
        return json.dumps({
            "local": "Sao Paulo", 
            "temperatura": 32, 
            "unidade": unidade})
    elif "porto alegre" in local.lower():
        return json.dumps({
            "local": "Porto Alegre",
            "temperatura": 25,
            "unidade": unidade})
    else:
        return json.dumps({
            "local": local,
            "temperatura": "unknown",
})
    
tools = [{
    "type": "function",
    "function": {
        "name": "obter_temperatura_atual",
        "description": "Obtem temperatura atual da cidade",
        "parameters": {
            "type": "object",
            "properties": {
                "local": {
                    "type": "string",
                    "description": "O nome da cidade. Exemplo: São Paulo",
                },
                "unidade": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["local"]
        },
    }
}]

In [6]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
    celsius = "celsius"
    fahrenheit = "fahrenheit"

class ObterTemperaturaAtual(BaseModel):
    """Obtem a temperatura atual de uma determinada localidade"""
    local: str = Field(description="O nome da cidade.", examples=["São Paulo", "Porto Alegre"])
    unidade: Optional[UnidadeEnum]

c:\Users\johns\OneDrive\Documentos\Projetos\Regdoor\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtual)
tool_temperatura

{'name': 'ObterTemperaturaAtual',
 'description': 'Obtem a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade.',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

#### Adicionando a funçao a Openai

In [8]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo")

resposta = chat.invoke("Qual a temperatura atual em Porto Alegre", functions=[tool_temperatura])
print(resposta)

content='' additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 105, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-c69ab0e8-8b13-43cf-a6cb-c261f4e2f12a-0' usage_metadata={'input_tokens': 105, 'output_tokens': 30, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


Simplificando o processo adicionando a função a reiz do nó

In [9]:
chat_function = chat.bind(functions=[tool_temperatura])
resposta = chat_function.invoke("Qual a temperatura em São Paulo?")
print(resposta)

content='' additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 103, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-ea0c774e-482a-456d-b8d9-7140858054aa-0' usage_metadata={'input_tokens': 103, 'output_tokens': 22, 'total_tokens': 125, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


#### Obrigando o modelo a usar a função

In [10]:
resposta = chat.invoke(
    "Qual a temperatura atual em Porto Alegre", 
    functions=[tool_temperatura],
    function_call= {"name": "ObterTemperaturaAtual"}
)
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 119, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1e3f240e-9b3f-4af3-acd9-5c6fc5e640f9-0', usage_metadata={'input_tokens': 119, 'output_tokens': 10, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Adicionando a uma Chain

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente grosso e arrogante chamado Isacc"),
    ("user", "{input}")
])

chain = prompt | chat.bind(functions=[tool_temperatura])

chain.invoke({"input": "Qual a temperatura em Floripa?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Florianópolis"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 118, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6631505e-3dc5-4d57-859a-d0abd577d2c3-0', usage_metadata={'input_tokens': 118, 'output_tokens': 24, 'total_tokens': 142, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})